## Projet Statistique


Ce projet a été fait par Enzo RABAHI, Léon DEBEVER, Lorenzo COUZINET et Quentin DOMON. Il consiste a utiliser les statistiques pour résoudre des problèmes de cryptographie. Ce projet va suivre la même structure que le document pdf qui nous a été fourni par Mme.CAUWET.

On va utiliser plusieurs bibliothèque python tel que numpy, et chisquare pour effectuer le test du chi-carré


In [1]:
import numpy as np
from scipy.stats import chisquare

On va commencer par importer les fichiers textes qui nous importe, c'est-à-dire les fichiers .txt

In [6]:
freq = np.loadtxt("frequence.txt")

## 1.1 Chiffrement par substitution et code de César

On utilise donc le fichier str.txt qui nous est fourni et que l'on va essayer de décrypter de manière itérative sachant qu'il y a 26 possibilités. On peut se permettre de faire une boucle qui va modifier le message en faisant une indentation de 1 à chaque fois.

In [10]:
f = open('str.txt', 'r')
contenu = f.read()
print(contenu)
f.close()

'gurpnrfnepvcurevfbarbsgurrneyvrfgxabjanaqfvzcyrfgpvcurefvgvfnglcrbsfhofgvghgvbapvcurevajuvpurnpuyrggrevagurcynvagrkgvffuvsgrqnpregnvaahzorebscynprfqbjagurnycunorg'



On peut voir ici à quoi ressemble le fichier texte en question. Il est normal qu'il nous parle pas puisqu'il est crypté

In [38]:
def cesar(txt, k):
    contenu = ""
    
    # 1. On ouvre le fichier
    f = open(txt, 'r')
    # 2. On lit TOUT le fichier d'un coup (pour avoir une chaine de caractères)
    texte_entier = f.read()
    f.close() # On ferme le fichier

    # 3. On parcourt chaque lettre
    for i in texte_entier:
        
        # LOGIQUE DE DÉCRYPTAGE :
        # On prend le code ASCII et on retire le décalage
        code_lettre = ord(i) - k 

        # GESTION DE LA BOUCLE (si on sort de l'alphabet)
        # Le code ASCII de 'a' est 97. Si en reculant on descend en dessous de 97, il faut rajouter 26pour repartir de la fin de l'alphabet ('z').
        if code_lettre < 97:
            code_lettre = code_lettre + 26
            
        contenu += chr(code_lettre)

    return contenu

In [39]:
for k in range (0,26,1):
    print(cesar('str.txt',k))

AgurpnrfnepvcurevfbarbsgurrneyvrfgxabjanaqfvzcyrfgpvcurefvgvfnglcrbsfhofgvghgvbapvcurevajuvpurnpuyrggrevagurcynvagrkgvffuvsgrqnpregnvaahzorebscynprfqbjagurnycunorgA$
@ftqomqemdoubtqdueazqarftqqmdxuqefwzaizmzpeuybxqefoubtqdeufuemfkbqaregnefufgfuazoubtqduzituotqmotxqffqduzftqbxmuzfqjfueeturfqpmoqdfmuzzgynqdarbxmoqepaizftqmxbtmnqf@#
?espnlpdlcntaspctdzypzqespplcwtpdevyzhylyodtxawpdentaspcdtetdlejapzqdfmdetefetzyntaspctyhstnsplnswpeepctyespawltyepietddstqepolnpceltyyfxmpczqawlnpdozhyesplwaslmpe?"
>dromkockbmszrobscyxoypdrookbvsocduxygxkxncswzvocdmszrobcsdsckdizoypcelcdsdedsyxmszrobsxgrsmrokmrvoddobsxdrozvksxdohdsccrspdonkmobdksxxewlobypzvkmocnygxdrokvzrklod>!
=cqnljnbjalryqnarbxwnxocqnnjaurnbctwxfwjwmbrvyunbclryqnabrcrbjchynxobdkbcrcdcrxwlryqnarwfqrlqnjlqunccnarwcqnyujrwcngcrbbqrocnmjlnacjrwwdvknaxoyujlnbmxfwcqnjuyqjknc= 
<bpmkimaizkqxpmzqawvmwnbpmmiztqmabsvwevivlaquxtmabkqxpmzaqbqaibgxmwnacjabqbcbqwvkqxpmzqvepqkpmikptmbbmzqvbpmxtiqvbmfbqaapqnbmlikmzbiqvvcujmzwnxtikmalwevbpmitxpijmb<
;aol

In [25]:
print(cesar('str.txt',13))

4thecaesarcipherisoneoftheearliestknownandsimplestciphersitisatypeofsubstitutioncipherinwhicheachletterintheplaintextisshiftedacertainnumberofplacesdownthealphabet4


Après vérification, on peut donc voir que le code césar à déchiffrer était d'un décalage de 14 vers la droite. 

On a parcouru l'entièreté des possibilités sachant qu'il y avait très peu (26). Cette méthode, certes efficace, ne peut être véritablement pris en compte de par la petitesse du nombre de cas.

On sait maintenant ce que ce code signifie, on va pouvoir passer au 1.3, c'est-à-dire au test du chi-carré

## 1.3 Test du CHI-carré


In [30]:


# 1. Chargement des fréquences théoriques (Source: 31)
# Attention : le fichier contient souvent des pourcentages (ex: 8.2 pour 8.2%).
# On divise par 100 pour avoir des probabilités entre 0 et 1.
freq_theorique_pourcentage = np.loadtxt('frequence.txt')
freq_theorique_proba = freq_theorique_pourcentage / 100

def compte_occurrences(texte):
    """
    Compte le nombre d'apparitions de chaque lettre (a-z).
    Retourne un tableau de 26 entiers.
    """
    compteur = np.zeros(26) # Tableau de 26 zéros
    for car in texte:
        index = ord(car) - ord('a')
        if 0 <= index <= 25:
            compteur[index] += 1
    return compteur

def chi2(fichier_chiffre):
    # Lecture du message chiffré
    f = open(fichier_chiffre, 'r')
    message_chiffre = f.read()
    f.close()
    
    print(f"Analyse du fichier '{fichier_chiffre}' ({taille_message} caractères)...\n")
    print(f"{'Clé':<5} | {'Chi-2 Stat':<12} | {'P-value':<12} | {'Début du texte décrypté'}")
    print("-" * 70)

    # On teste les 26 clés possibles (Source: 73)
    for k in range(26):
        # 1. Déchiffrer avec la clé k
        texte_candidat = cesar(message_chiffre, k)
        
        # 2. Déterminer les fréquences observées (compte réel des lettres)
        f_obs = compte_occurrences(texte_candidat)
        
        # 3. Calculer les fréquences théoriques attendues pour un texte de cette longueur, donc le nombre total de lettre * sa propre proba.
        f_exp = freq_theorique_proba * taille_message
        
        # 4. Test du Chi-2 
        # On compare le tableau des observés avec le tableau des fréquences
        chi2_stat, p_value = chisquare(f_obs, f_exp=f_exp)
        
        resultats.append((k, chi2_stat, p_value, texte_candidat[:30]))
        
        # Affichage formaté pour voir ce qu'il se passe
        print(f"{k:<5} | {chi2_stat:<12.2f} | {p_value:.2e}    | {texte_candidat[:30]}...")

    # --- Analyse des résultats ---
    # On cherche la p-value la plus élevée (ou le Chi2 le plus bas)
    # On s'intéresse aux p-values > 5% (0.05) pour ne pas rejeter H0
    
    print("-" * 70)
    meilleure_cle = max(resultats, key=lambda x: x[2]) # On prend le max de p-value
    
    print(f"\nLA MEILLEURE CLÉ SEMBLE ÊTRE : {meilleure_cle[0]}")
    print(f"Message décrypté : {cesar(message_chiffre, meilleure_cle[0])}")

chi2('str.txt')

OSError: [Errno 22] Invalid argument: "'gurpnrfnepvcurevfbarbsgurrneyvrfgxabjanaqfvzcyrfgpvcurefvgvfnglcrbsfhofgvghgvbapvcurevajuvpurnpuyrggrevagurcynvagrkgvffuvsgrqnpregnvaahzorebscynprfqbjagurnycunorg'\n"

In [35]:
# 1. Chargement des fréquences
freq_theorique_pourcentage = np.loadtxt('frequence.txt')
freq_theorique_proba = freq_theorique_pourcentage / 100


def compte_occurrences(texte):
    compteur = np.zeros(26)
    for car in texte:
        index = ord(car) - ord('a')
        if 0 <= index <= 25:
            compteur[index] += 1
    return compteur

def chi2(fichier_chiffre):
    # --- 1. Lecture et Nettoyage du fichier ---
    with open(fichier_chiffre, 'r') as f:
        brut = f.read()
    
    # IMPORTANT : Ton fichier str.txt contient des guillemets et des sauts de ligne.
    # Il faut les nettoyer pour ne garder que le texte chiffré.
    message_chiffre = brut.replace("'", "").replace("\n", "").strip()
    
    taille_message = len(message_chiffre)
    resultats = [] # On initialise la liste pour stocker les scores
    
    print(f"Analyse de '{fichier_chiffre}' ({taille_message} caractères utiles)...\n")
    print(f"{'Clé':<5} | {'Chi-2 Stat':<12} | {'P-value':<12} | {'Début du texte décrypté'}")
    print("-" * 70)

    # --- 2. Boucle sur les clés ---
    for k in range(26):
        # On envoie le TEXTE à la fonction cesar (pas le nom de fichier)
        texte_candidat = cesar(message_chiffre, k)
        
        f_obs = compte_occurrences(texte_candidat)
        
        # Calcul des fréquences attendues (taille du texte * proba théorique)
        f_exp = freq_theorique_proba * taille_message
        
        # Test du Chi-2
        chi2_stat, p_value = chisquare(f_obs, f_exp=f_exp)
        
        resultats.append((k, chi2_stat, p_value, texte_candidat[:30]))
        
        print(f"{k:<5} | {chi2_stat:<12.2f} | {p_value:.2e}    | {texte_candidat[:30]}...")

    # --- 3. Meilleur résultat ---
    print("-" * 70)
    meilleure_cle = max(resultats, key=lambda x: x[2]) # Max P-value
    
    print(f"\nLA MEILLEURE CLÉ SEMBLE ÊTRE : {meilleure_cle[0]}")
    print(f"P-value : {meilleure_cle[2]}")
    # On déchiffre une dernière fois avec la bonne clé pour l'afficher
    print(f"Message décrypté : {cesar(message_chiffre, meilleure_cle[0])}")

# Appel de la fonction
chi2('str.txt')

Message chargé et nettoyé (162 lettres) : gurpnrfnepvcurevfbarbsgurrneyv...

Clé   | Chi-2      | P-value    | Extrait décrypté
-----------------------------------------------------------------


ValueError: For each axis slice, the sum of the observed frequencies must agree with the sum of the expected frequencies to a relative tolerance of 1e-08, but the percent differences are:
0.0011899999999999942